In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model:
                roto_extractor has       1750 params (~ 50.7) %
               trans_extractor has       1699 params (~ 49.3) %
total params:                 3449
cam2cam estimation => adding 3449 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@12.05.2021-10:19:56
#[do_train]: cam2cam loss weights: geo: 0.0, trans: 0.0, proj: 1.0
#[do_train]: epoch    0 has started!
failed loading /home/stefano/Scuola/tud/_classes/4/thesis/data/processed/S1/Directions-1/imageSequence/54138969/img_000001.jpg from disk -> sleeping 0.1 minutes ...
... AWAKE! retrying (for the # 2 time)
failed loading /home/stefano/Scuola/tud/_classes/4/thesis/data/processed/S1/Directions-1/imageSequence/54138969/img_000001.jpg from disk -> sleeping 0.1 minutes ...


OSError: fix that cluster !!!